In [ ]:
import json
import os
from ovejero import model_trainer

In [ ]:
# Start the base dictionary
training_params = {}

# Probably the most important parameter - the type of BNN we want to train
training_params['bnn_type'] = 'full'

# First the optimization specific parameters
training_params['batch_size'] = 128
# You can specify a large number of epochs. The model saves each epoch
training_params['n_epochs'] = 80
training_params['learning_rate'] = 1e-5
training_params['decay'] = 0.000003
training_params['kernel_regularizer'] = 1e-5
training_params['dropout_regularizer'] = 1e-6

# Now set the parameters that will point us to the tf_record
# For this example we want to set the path to the test files
training_params['root_path'] = os.getcwd()[:-7]+'datasets/train/'
training_params['tf_record_path'] = 'tf_record_train'

# Set the final list of parameters we want to use for analysis
training_params['final_params'] = ['external_shear_e1','external_shear_e2','lens_mass_center_x',
                                   'lens_mass_center_y','lens_mass_e1','lens_mass_e2','lens_mass_gamma',
                                   'lens_mass_theta_E_log']
# Using the same ordering as final_params, we also want to set the flip_pairs. These are all params with
# excentricities.
training_params['flip_pairs'] = [[0,1],[4,5],[0,1,4,5]]
# The dimensions of the images
training_params['img_dim'] = 128

# The path to load/save the model weights to for this configuration.
training_params['model_weights'] = os.getcwd()[:-7]+'models/'+'t2.h5'

# Where to save the tensorboard logs to
training_params['tensorboard_log_dir'] = os.getcwd()[:-7]+'logs/'+'t2.log'

# Set the random seed at train time for reproducibility!
training_params['random_seed'] = 1138

# Make sure we've set the right number of training parameters (not a full proof check)
if len(training_params) < 15:
    raise RuntimeError('Missing something in training_params!')
elif len(training_params) > 15:
    raise RuntimeError('Too much stuff in training_params!')
else:
    print('All set!')

In [ ]:
# Make sure we're happy with the flip pairs!
pair_index = 1
for flip_pair in training_params['flip_pairs']:
    print('Pair %d'%(pair_index))
    for fp in flip_pair:
        print(training_params['final_params'][fp])
    pair_index += 1

Now we want to set up the **val_params**

In [ ]:
# Start the base dictionary
val_params = {}

# All we really need to set here is the path to the validation data and the desired output path to the TFRecord.
# The rest can just be pulled from the training_params.
val_params['root_path'] = os.getcwd()[:-7]+'datasets/train/'
val_params['tf_record_path'] = 'tf_record_val'

# Make sure we've set the right number of validation parameters (not a full proof check)
if len(val_params) < 2:
    raise RuntimeError('Missing something in training_params!')
elif len(val_params) > 2:
    raise RuntimeError('Too much stuff in training_params!')
else:
    print('All set!')

Now we want to repeat the same for the **dataset_params**

In [ ]:
# Start the base dictionary
dataset_params = {}

# First set path for the old and new lensing parameters. If you don't mind rewriting the old file, these can be the
# same.
dataset_params['lens_params_path'] = 'metadata.csv'
dataset_params['new_param_path'] = 'new_metadata.csv'
# Also set where to save the normalization weights to
dataset_params['normalization_constants_path'] = 'norms.npy'

# Set the lens parameters to be pulled from metadata.csv.
dataset_params['lens_params'] = ['external_shear_gamma_ext','external_shear_psi_ext','lens_mass_center_x',
                                 'lens_mass_center_y','lens_mass_e1','lens_mass_e2','lens_mass_gamma',
                                 'lens_mass_theta_E']

# List of parameters that need to be converted to log space
dataset_params['lens_params_log'] = ['lens_mass_theta_E']

# List of parameters that need to be converted from ratio and angle to excentricities
dataset_params['ratang'] = {}
dataset_params['ratang']['ratang_parameter_prefixes'] = ['external_shear']
dataset_params['ratang']['ratang_params_rat'] = ['external_shear_gamma_ext']
dataset_params['ratang']['ratang_params_ang'] = ['external_shear_psi_ext']

# Make sure we've set the right number of dataset parameters (not a full proof check)
if len(dataset_params) < 6:
    raise RuntimeError('Missing something in dataset_params!')
elif len(dataset_params) > 6:
    raise RuntimeError('Too much stuff in dataset_params!')
elif len(dataset_params['ratang']) != 3:
    raise RuntimeError('Ratio and angle parameters are off!')
else:
    print('All set!')

Finally, we can combine this into our configuration dictionairy and write it to a json file!

In [ ]:
json_path = os.getcwd()+'/t2.json'
config_dict = {'training_params':training_params, 'validation_params':val_params,'dataset_params':dataset_params}
model_trainer.config_checker(config_dict)
with open(json_path,'w') as json_f:
    json.dump(config_dict,json_f,indent=4)